# Cours TAL – Labo 4
Nathan Gonzalez Montes et Vincent Guidoux

## Exercice 1 - exécuter la NER dans NLTK

### En utilisant nltk.ne_chunk (voir livre NLTK p. 283 - 4),extraire les entités nommées les plus fréquentes  d’un texte de votre choix

In [1]:
import sys
!{sys.executable} -m pip install numpy

In [2]:
import nltk
import os, codecs
from nltk.tokenize import sent_tokenize
from nltk.tag import pos_tag
from nltk import ne_chunk
from nltk import FreqDist
FreqDist
from urllib import request
ne_chunk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nortalle\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Nortalle\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Nortalle\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [3]:
url1 = "http://www.gutenberg.org/files/18488/18488.txt"

response = request.urlopen(url1)
raw = response.read().decode('utf8')
raw = raw[4340:489067]

In [4]:
sentences = nltk.sent_tokenize(raw)

sentences = [nltk.word_tokenize(sent) for sent in sentences]

len(sentences)

6701

In [ ]:
sentences = [nltk.pos_tag(sent) for sent in sentences]
    
len(sentences)

6701

In [ ]:
import numpy
from nltk import FreqDist

trees = []

for sentence in sentences:
    for chunk in ne_chunk(sentence):
        if hasattr(chunk, 'label'):
            trees.append((chunk.label(), ' '.join(c[0] for c in chunk)))

In [ ]:
FreqDist(trees).most_common(10)

[(('PERSON', 'Priscilla'), 227),
 (('PERSON', 'Farwell'), 194),
 (('GPE', 'Priscilla'), 193),
 (('PERSON', 'Ledyard'), 106),
 (('PERSON', 'Master Farwell'), 64),
 (('ORGANIZATION', 'Boswell'), 61),
 (('ORGANIZATION', 'Travers'), 59),
 (('PERSON', 'Nathaniel'), 55),
 (('GPE', 'Kenmore'), 55),
 (('PERSON', 'Margaret'), 52)]

### En vous inspirant du POS de Stanford, utiliser leur NER (https://nlp.stanford.edu/software/CRF-NER.html) et la classe StanfordNERTagger de NLTK, pour extraire les entités nommées du même texte


#### Création du fichier pour qu'il soit utilisé par POS de STanford

In [ ]:
filename = "gutenberg_18488.txt"

tokens = nltk.word_tokenize(raw)

if os.path.exists(filename): 
    os.remove(filename)
    
fd = codecs.open(filename, 'a', 'utf8')

try:
    for i in tokens:
        fd.write(i)
        fd.write(" ")
finally:
    fd.close()

In [ ]:
from nltk.tag import StanfordNERTagger
st = StanfordNERTagger('stanford/classifiers/english.all.3class.distsim.crf.ser.gz', 
                       'stanford/stanford-ner.jar', 
                       encoding='utf-8')
sentences = nltk.sent_tokenize(raw)

sentences = [nltk.word_tokenize(sent) for sent in sentences]




In [ ]:
for sentence in sentences:
    classified_text = st.tag(sentence)
    print(classified_text)

[('CHAPTER', 'O'), ('I', 'O'), ('Priscilla', 'PERSON'), ('Glenn', 'PERSON'), ('stood', 'O'), ('on', 'O'), ('the', 'O'), ('little', 'O'), ('slope', 'O'), ('leading', 'O'), ('down', 'O'), ('from', 'O'), ('the', 'O'), ('farmhouse', 'O'), ('to', 'O'), ('the', 'O'), ('spring', 'O'), ('at', 'O'), ('the', 'O'), ('bottom', 'O'), ('of', 'O'), ('the', 'O'), ('garden', 'O'), (',', 'O'), ('and', 'O'), ('lifted', 'O'), ('her', 'O'), ('head', 'O'), ('as', 'O'), ('a', 'O'), ('young', 'O'), ('deer', 'O'), ('does', 'O'), ('when', 'O'), ('it', 'O'), ('senses', 'O'), ('something', 'O'), ('new', 'O'), ('or', 'O'), ('dangerous', 'O'), ('.', 'O')]
[('Suddenly', 'O'), (',', 'O'), ('and', 'O'), ('entirely', 'O'), ('subconsciously', 'O'), (',', 'O'), ('she', 'O'), ('felt', 'O'), ('her', 'O'), ('kinship', 'O'), ('with', 'O'), ('life', 'O'), (',', 'O'), ('her', 'O'), ('relation', 'O'), ('to', 'O'), ('the', 'O'), ('lovely', 'O'), ('May', 'O'), ('day', 'O'), ('which', 'O'), ('was', 'O'), ('more', 'O'), ('like', 'O

[('The', 'O'), ('first', 'O'), ('child', 'O'), ('was', 'O'), ('as', 'O'), ('dark', 'O'), ('as', 'O'), ('a', 'O'), ('little', 'O'), ('Indian', 'O'), (',', 'O'), ('``', 'O'), ('so', 'O'), ('dark', 'O'), (',', 'O'), ("''", 'O'), ('explained', 'O'), ('Nathaniel', 'PERSON'), (',', 'O'), ('``', 'O'), ('that', 'O'), ('she', 'O'), ('would', 'O'), ('have', 'O'), ('been', 'O'), ('welcome', 'O'), ('in', 'O'), ('any', 'O'), ('house', 'O'), ('on', 'O'), ('a', 'O'), ('New', 'O'), ("Year's", 'O'), ('Day', 'O'), ('.', 'O'), ("''", 'O')]
[('She', 'O'), ('lasted', 'O'), ('but', 'O'), ('a', 'O'), ('year', 'O'), (',', 'O'), ('and', 'O'), (',', 'O'), ('while', 'O'), ('she', 'O'), ('was', 'O'), ('a', 'O'), ('regret', 'O'), (',', 'O'), ('she', 'O'), ('had', 'O'), ('been', 'O'), ('tolerated', 'O'), ('as', 'O'), ('an', 'O'), ('attempt', 'O'), (',', 'O'), ('at', 'O'), ('least', 'O'), (',', 'O'), ('in', 'O'), ('the', 'O'), ('right', 'O'), ('direction', 'O'), ('.', 'O')]
[('Then', 'O'), ('came', 'O'), ('the', 'O'

[('And', 'O'), ('in', 'O'), ('her', 'O'), ('heart', 'O'), ('of', 'O'), ('hearts', 'O'), ('Mary', 'PERSON'), ('McAdam', 'PERSON'), ('knew', 'O'), ('this', 'O'), ('to', 'O'), ('be', 'O'), ('true', 'O'), ('.', 'O')]
[('The', 'O'), ('time', 'O'), ('would', 'O'), ('come', 'O'), ('to', 'O'), ('her', 'O'), (',', 'O'), ('as', 'O'), ('it', 'O'), ('had', 'O'), ('to', 'O'), ('all', 'O'), ('Kenmore', 'LOCATION'), ('mothers', 'O'), (',', 'O'), ('when', 'O'), ('she', 'O'), ('would', 'O'), ('have', 'O'), ('to', 'O'), ('acknowledge', 'O'), ('that', 'O'), ('by', 'O'), ('the', 'O'), ('power', 'O'), ('of', 'O'), ('the', 'O'), ('``', 'O'), ('lure', 'O'), ("''", 'O'), ('were', 'O'), ('her', 'O'), ('boys', 'O'), ('to', 'O'), ('be', 'O'), ('tested', 'O'), ('.', 'O')]
[('But', 'O'), ('Priscilla', 'PERSON'), ('at', 'O'), ('Lonely', 'O'), ('Farm', 'O'), ('showed', 'O'), ('a', 'O'), ('hardened', 'O'), ('disregard', 'O'), ('of', 'O'), ('her', 'O'), ('state', 'O'), ('.', 'O')]
[('She', 'O'), ('persisted', 'O'), ('

### Comparez la liste des NE les plus fréquentes et leurs types reconnus

## Exercice 2

[FreqDist](https://stackoverflow.com/questions/4634787/freqdist-with-nltk)